<a href="https://colab.research.google.com/github/SemihDemir13/chatbot-project-data/blob/main/V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#1
print("Gerekli kütüphaneler kuruluyor...")
!pip install -q -U transformers huggingface_hub accelerate sentence-transformers torch langchain langchain-community langchain-huggingface chromadb pandas
print("Kurulum tamamlandı.")

In [ ]:
#2
from huggingface_hub import login

# Token'ını girdikten sonra 'Login successful' yazısını görmelisin.
login()

In [ ]:
#3
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

print("Model yükleniyor... (Biraz sabır)")

model_id = "google/gemma-2b-it"

try:
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    # Colab GPU'su kullanılsın diye device_map="auto" diyoruz
    model = AutoModelForCausalLM.from_pretrained(
        model_id,
        device_map="auto",
        torch_dtype="auto"
    )

    # Text generation pipeline'ı oluşturuyoruz
    llm_pipeline = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=250 # Cevabın uzunluğu
    )
    print("✅ Model başarıyla yüklendi!")

except Exception as e:
    print(f"❌ HATA: Model yüklenemedi. Lütfen HÜCRE 2'de token girdiğinizden ve modelin lisansını kabul ettiğinizden emin olun.\nHata: {e}")


In [ ]:
#4
import pandas as pd
import sys

print("CSV dosyası okunuyor ve veri formatlanıyor...")

try:
    df = pd.read_csv('netflix_titles.csv')
    df = df.fillna('Bilinmiyor')

    dokumanlar = []

    # Tüm veriyi alıyoruz
    print(f"Toplam {len(df)} adet kayıt işleniyor...")

    for index, row in df.iterrows():
        # Formatı sadeleştirdik. Başlık en üstte.
        metin = (
            f"Başlık: {row['title']}\n"
            f"Yönetmen: {row['director']}\n"
            f"Yıl: {row['release_year']}\n"
            f"Tür: {row['listed_in']}\n"
            f"Özet: {row['description']}\n"
            f"Detay: Bu film {row['release_year']} yılında yayınlanmıştır. {row['type']} kategorisindedir."
        )
        dokumanlar.append(metin)

    print(f"✅ {len(dokumanlar)} adet içerik başarıyla hazırlandı.")

except FileNotFoundError:
    print("❌ HATA: Dosya yok.")

In [ ]:
#5
from langchain_community.vectorstores import Chroma
from langchain_huggingface import HuggingFaceEmbeddings

print("Vektör veritabanı (Çok Dilli) oluşturuluyor...")

# DEĞİŞİKLİK BURADA:
# Artık Türkçe-İngilizce eşleştirmesi yapabilen 'multilingual' modeli kullanıyoruz.
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")

# ChromaDB veritabanını kur
# Eğer hata alırsan önceki veritabanını temizlemek gerekebilir ama genellikle üzerine yazar.
vektor_veritabani = Chroma.from_texts(dokumanlar, embedding=embedding_model)

print("✅ Çok dilli veritabanı hazır! Artık Türkçe sorguları İngilizce verilerle eşleştirebilirim.")

In [ ]:
import re
import random

def rag_yanit(soru, top_k=3):
    # --- HAZIRLIK ---
    etkisiz_kelimeler = ["filmi", "film", "öner", "bana", "hakkında", "olan", "yapımı", "yönetmen", "oyuncu", "kimdir", "listesi", "var", "mı", "mu", "bir", "söyle", "istiyorum", "tavsiye", "et"]
    temiz_soru = soru.lower()
    for kelime in etkisiz_kelimeler:
        temiz_soru = temiz_soru.replace(f" {kelime} ", " ").replace(f" {kelime}", "").replace(f"{kelime} ", "")
    anahtar_kelime = temiz_soru.strip()
    anahtar_kelime_bitisik = anahtar_kelime.replace(" ", "")

    secilen_film_bilgisi = {} # Bulunan filmi burada tutacağız
    kaynak_yontem = ""

    # --- 1. ADIM: FİLTRELEME (PANDAS) ---
    yil_tespiti = re.search(r'(\d{4})', soru)
    uygun_filmler = pd.DataFrame()

    # A) YIL VARSA
    if yil_tespiti:
        aranan_yil = int(yil_tespiti.group(1))
        kaynak_yontem = f"YIL FİLTRESİ ({aranan_yil})"
        uygun_filmler = df[df['release_year'] == aranan_yil]

    # B) İSİM VARSA
    elif len(anahtar_kelime) > 2:
        m_dir = df['director'].str.contains(anahtar_kelime, case=False, na=False) | df['director'].str.contains(anahtar_kelime_bitisik, case=False, na=False)
        m_cast = df['cast'].str.contains(anahtar_kelime, case=False, na=False) | df['cast'].str.contains(anahtar_kelime_bitisik, case=False, na=False)
        uygun_filmler = df[m_dir | m_cast]
        if not uygun_filmler.empty:
            kaynak_yontem = f"İSİM FİLTRESİ ('{anahtar_kelime}')"

    # --- 2. ADIM: VERİYİ SEÇME ---
    if not uygun_filmler.empty:
        # Rastgele 1 film seç
        secilen = uygun_filmler.sample(n=1).iloc[0]
        secilen_film_bilgisi = {
            "title": secilen['title'],
            "year": secilen['release_year'],
            "desc": secilen['description']
        }
        print(f"\n[SİSTEM: {kaynak_yontem} bulundu: {secilen['title']}]")

    # C) HİÇBİRİ YOKSA -> VEKTÖR ARAMASI
    else:
        kaynak_yontem = "VEKTÖR ARAMASI"
        if len(anahtar_kelime) > 3 and not yil_tespiti:
             print(f"\n[SİSTEM: '{anahtar_kelime}' bulunamadı. Konuya göre aranıyor...]")

        # Vektörden en alakalı 1 taneyi alıyoruz (Kafası karışmasın diye)
        docs = vektor_veritabani.similarity_search(soru, k=1)
        if docs:
            doc = docs[0]
            lines = doc.page_content.split('\n')

            # Metinden veriyi ayıkla
            title = "Bilinmiyor"
            desc = "Açıklama yok."
            year = "???"

            for line in lines:
                if "Başlık:" in line: title = line.replace("Başlık: ", "").strip()
                if "Özet:" in line or "Description:" in line: desc = line.split(":", 1)[1].strip()
                if "Yıl:" in line: year = line.replace("Yıl: ", "").strip()

            secilen_film_bilgisi = {"title": title, "year": year, "desc": desc}
            print(f"\n[SİSTEM: Vektörden bulundu: {title}]")

    print("-" * 40)

    # --- 3. ADIM: SADECE ÇEVİRİ PROMPTU ---
    # Eğer hiç film bulunamadıysa
    if not secilen_film_bilgisi:
        return "Üzgünüm, kriterlerinize uygun bir film bulamadım."

    ingilizce_ozet = secilen_film_bilgisi['desc']

    # Modele ÇOK BASİT bir görev veriyoruz: Sadece çevir.
    prompt = f"""Translate the following movie description from English to Turkish.

English: {ingilizce_ozet}

Turkish:"""

    # Model çeviriyi yapsın
    try:
        # max_new_tokens'ı kısalttık ki saçmalamasın
        sonuc = llm_pipeline(prompt, return_full_text=False, max_new_tokens=100)[0]["generated_text"]
        turkce_ozet = sonuc.strip()
        # Eğer model prompt'u tekrar ederse temizle
        if "Turkish:" in turkce_ozet:
            turkce_ozet = turkce_ozet.split("Turkish:")[-1].strip()
    except:
        turkce_ozet = ingilizce_ozet # Hata olursa orijinalini bas

    # --- 4. ADIM: CEVABI BİZ BİRLEŞTİRİYORUZ ---
    # Formatı Python ile yapıyoruz, böylece yapay zeka formatı bozamaz.
    final_cevap = f"Önerim: {secilen_film_bilgisi['title']} ({secilen_film_bilgisi['year']})\nKonusu: {turkce_ozet}"

    return final_cevap

In [ ]:
print("\n================================================")
print("🎬 NETFLIX GURUSU (Çıkmak için 'çıkış' yazın)")
print("================================================\n")

while True:
    soru = input("Soru sor: ")
    if soru.lower() in ['çıkış', 'exit', 'quit']:
        print("İyi seyirler 👋")
        break

    print("🔎 Araştırıyorum...")
    cevap = rag_yanit(soru)
    print(f"\n🤖 AI: {cevap}\n")
    print("-" * 40)
